# **Practical assignment for Topic 8**

In the assignment, you have to use the dataset `auto-lpkm.tsv` given in the lecture and you may reuse the given code.

Your assignment consists of **three tasks**:

**Task 1**<br>
Create a new function called `SBS` and implement the SBS algorithm (as explained in the lecture) by copying the given implementation of SFS and modifying it. When you run it, it should have exactly the same number of iterations as SFS. Verify its correctness by checking if it returns the overall best coefficient of determination as `0.8646172504920925`.

**Task 2**<br>
Modify your code further to create a plot from the SBS iteration results, where X axis is the number of selected features and the Y axis is the best MSE of that iteration. Write a short analysis of the plot (what can you conclude from it?).

**Task 3**<br>
Replace the Linear Regression model with KNN model (with k = 5) and run SFS and SBS with it. Look at the returned MSE and R2 values and the new plot. Compare the results to those of Linear Regression and write a short analysis of the returned values and plots, highlighting the main aspects. (Note that your notebook should now have two plots - one from Linear Regression (Task 2) and one from KNN (Task 3).)

In [ ]:
# write your code here

---
**After the tasks are done, submit this file. Do not clear it's output - all print-outs and diagrams should be left in the file.**